In [21]:
import tensorflow as tf
import zipfile
tfkl = tf.keras.layers
import csv
import numpy as np
import pandas as pd
import xport
import os as os
import matplotlib.pyplot as plt
#from keras.layers import Input, Embedding, Flatten, Dense, BatchNormalization, Dropout, concatenate
#from keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Load in Data

In [33]:
# loading in USMDB data
data = []
ages = []
states = []
genders = []

with open("../data/usmdb/usmdb.csv", "r") as file:
    reader = csv.reader(file,delimiter=',')
    for row_index, row in enumerate(reader):
        if row_index == 0:
            print(row)
        if row_index >= 1:
            state, gender, year, age, rate = row
            year = int(year)
            try:
                age = int(age)
            except:
                age = -1
            if state not in states:
                states.append(state)
            state = states.index(state)
            if gender not in genders:
                genders.append(gender)
            gender = genders.index(gender)
            try:
                rate = float(rate)
            except:
                rate = -1
            if rate > 1:
                rate = 1
            # get rid of years, ages, not in health data and other cleaning
            if age != -1 and rate != -1 and age <= 99 and age >=18 and year >= 1993:
                data.append([state, gender, year, age, rate])

data = np.array(data)

['PopName', 'Sex', 'Year', 'Age', 'mx']


In [41]:
print(data[:,0])
print(data)
print(states)
print(genders)

[ 0.  0.  0. ... 49. 49. 49.]
[[0.0000e+00 0.0000e+00 1.9930e+03 1.8000e+01 2.7000e-04]
 [0.0000e+00 0.0000e+00 1.9930e+03 1.9000e+01 0.0000e+00]
 [0.0000e+00 0.0000e+00 1.9930e+03 2.0000e+01 1.0700e-03]
 ...
 [4.9000e+01 1.0000e+00 2.0200e+03 9.7000e+01 3.8917e-01]
 [4.9000e+01 1.0000e+00 2.0200e+03 9.8000e+01 4.2072e-01]
 [4.9000e+01 1.0000e+00 2.0200e+03 9.9000e+01 4.5292e-01]]
['AK', 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
['f', 'm']


In [63]:
df = pd.read_csv("../data/brfss/brfss_agg.csv", index_col=0)

Last minute cleaning. Will put chunk below into cleaning file soon.

In [64]:
# removing missing - should add this to cleaning dataset in the future
index_missing = df[ (df['sex'] == '7')].index
df.drop(index_missing, inplace=True)

# Mapping from FIPS code to state abbreviation
fips_to_abbreviation = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE',
    13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 21: 'KY',
    22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO',
    30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY', 37: 'NC',
    38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC', 46: 'SD',
    47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI',
    56: 'WY'
}

# Convert FIPS codes to state abbreviations
df['state'] = df['state'].map(fips_to_abbreviation)

print(df)


      state    age     sex  year        bmi
0        AL  18-24  female  1993  21.704250
1        AL  18-24  female  1994  23.100608
2        AL  18-24  female  1995  23.313510
3        AL  18-24  female  1996  24.502406
4        AL  18-24  female  1997  23.347746
...     ...    ...     ...   ...        ...
38849    WY    80+    male  2017  27.069832
38850    WY    80+    male  2018  26.646026
38851    WY    80+    male  2019  26.365848
38852    WY    80+    male  2020  26.726692
38853    WY    80+    male  2021  26.220133

[37570 rows x 5 columns]


In [65]:
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['state'] = df['state'].astype('category')
df['state'] = df['state'].apply(lambda x: states.index(x))
df['age'] = df['age'].astype('category')
df['sex'] = df['sex'].map({'female': 0, 'male': 1})
print(df['state'])

health_data = df.to_numpy()

print(health_data)
print(health_data.shape)

0         1
1         1
2         1
3         1
4         1
         ..
38849    49
38850    49
38851    49
38852    49
38853    49
Name: state, Length: 37570, dtype: category
Categories (49, int64): [0, 1, 3, 2, ..., 46, 48, 47, 49]
[[1.0 '18-24' 0 1993 21.70425030465645]
 [1.0 '18-24' 0 1994 23.100607781605586]
 [1.0 '18-24' 0 1995 23.313510430118065]
 ...
 [49.0 '80+' 1 2019 26.36584752016349]
 [49.0 '80+' 1 2020 26.72669220177878]
 [49.0 '80+' 1 2021 26.220133114156347]]
(37570, 5)


# Train no covariate model

In [25]:
# training and test sets 
training_index = np.logical_and(data[:, 2] >= 1998, data[:, 2] <= 2010)
training_data = data[training_index, :]
print(training_data.shape)

test_index = np.logical_and(data[:, 2] > 2010, data[:, 2] <= 2015)
test_data = data[test_index, :]

(107900, 5)


In [26]:
training_data = tf.convert_to_tensor(training_data)
test_data = tf.convert_to_tensor(test_data)
# cast tensor to type float32
training_data = tf.cast(training_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)
num_train = training_data.shape[0]
num_test = test_data.shape[0]

In [27]:
# define function to fetch and process data entries from training or test data 
def get_data(index, mode):
    if mode == "train":
        # randomly selects index from training data between 0 and num_train
        rand_index = tf.random.uniform([],minval=0, maxval=num_train, dtype=tf.int32) 
        entry = training_data[rand_index, :]
    elif mode == "not_random":
        # selects specified index from test data 
        entry = test_data[index, :]
    else: 
        # for any other value of mode, randomly selects index from test
        rand_index = tf.random.uniform([],minval=0, maxval=num_test, dtype=tf.int32)
        entry = test_data[rand_index, :]
    state, gender, year, age, rate = entry[0], entry[1], entry[2], entry[3], entry[4]
    year = (year - 1998)/21
    age = tf.cast(age, tf.int32)
    state = tf.cast(state, tf.int32)
    gender = tf.cast(gender, tf.int32)
    year = tf.reshape(year, [1])
    age = tf.reshape(age, [1])
    state = tf.reshape(state, [1])
    gender = tf.reshape(gender, [1])
    rate = tf.reshape(rate, [1])
    return (year, age, state, gender), rate

In [28]:
# use get_data function to set up training and test tensorflow datasets 
dataset_train = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_train = dataset_train.repeat()
dataset_train = dataset_train.map(lambda x: get_data(x, mode="train"), num_parallel_calls=4)
dataset_train = dataset_train.batch(256)
dataset_train = dataset_train.prefetch(buffer_size=512)

dataset_test = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_test = dataset_test.repeat()
dataset_test = dataset_test.map(lambda x: get_data(x, mode="test"), num_parallel_calls=4)
dataset_test = dataset_test.batch(256)
dataset_test = dataset_test.prefetch(buffer_size=512)

dataset_test2 = tf.data.Dataset.from_tensor_slices(np.arange(68000))
dataset_test2 = dataset_test2.map(lambda x: get_data(x, mode="not_random"), num_parallel_calls=4)
dataset_test2 = dataset_test2.batch(256)
dataset_test2 = dataset_test2.prefetch(buffer_size=512)

In [29]:
# defining inputs 
year = tfkl.Input(shape=(1,), dtype='float32', name='Year')
age =  tfkl.Input(shape=(1,), dtype='int32', name='Age')
state = tfkl.Input(shape=(1,), dtype='int32', name='State')
gender = tfkl.Input(shape=(1,), dtype='int32', name='Gender')

# defining embedding layers 
age_embed = tfkl.Embedding(input_dim=100, output_dim=5, input_length=1, name='Age_embed')(age)
age_embed = tfkl.Flatten()(age_embed)

gender_embed = tfkl.Embedding(input_dim=2, output_dim=5, input_length=1, name='Gender_embed')(gender)
gender_embed = tfkl.Flatten()(gender_embed)

state_embed = tfkl.Embedding(input_dim=50, output_dim=5, input_length=1, name='State_embed')(state)
state_embed = tfkl.Flatten()(state_embed)

# create feature vector that concatenates all inputs 
x = tfkl.Concatenate()([year, age_embed, gender_embed, state_embed])
x1 = x

# setting up middle layers 
x = tfkl.Dense(128, activation='tanh')(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dropout(0.05)(x)

x = tfkl.Dense(128, activation='tanh')(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dropout(0.05)(x)

x = tfkl.Dense(128, activation='tanh')(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dropout(0.05)(x)

x = tfkl.Dense(128, activation='tanh')(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dropout(0.05)(x)

# setting up output layer 
x = tfkl.Concatenate()([x1, x])
x = tfkl.Dense(128, activation='tanh')(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dropout(0.05)(x)
x = tfkl.Dense(1, activation='sigmoid', name='final')(x)

# creating the model 
model = tf.keras.Model(inputs=[year, age, state, gender], outputs=[x])

In [30]:
# compiling the model
model.compile(loss='mse', optimizer='adam')

In [31]:
model.fit(dataset_train, steps_per_epoch=1000, validation_data=dataset_test, validation_steps=500, epochs=9, verbose=2)

Epoch 1/9
1000/1000 - 16s - loss: 0.0135 - val_loss: 5.4982e-04 - 16s/epoch - 16ms/step
Epoch 2/9
1000/1000 - 13s - loss: 9.2730e-04 - val_loss: 1.6174e-04 - 13s/epoch - 13ms/step
Epoch 3/9
1000/1000 - 14s - loss: 4.1908e-04 - val_loss: 1.1740e-04 - 14s/epoch - 14ms/step
Epoch 4/9
1000/1000 - 13s - loss: 2.5593e-04 - val_loss: 1.1861e-04 - 13s/epoch - 13ms/step
Epoch 5/9
1000/1000 - 14s - loss: 1.8291e-04 - val_loss: 1.7523e-04 - 14s/epoch - 14ms/step
Epoch 6/9
1000/1000 - 14s - loss: 1.2961e-04 - val_loss: 6.0400e-05 - 14s/epoch - 14ms/step
Epoch 7/9
1000/1000 - 14s - loss: 9.9356e-05 - val_loss: 3.5780e-05 - 14s/epoch - 14ms/step
Epoch 8/9
1000/1000 - 14s - loss: 9.1452e-05 - val_loss: 8.3763e-05 - 14s/epoch - 14ms/step
Epoch 9/9
1000/1000 - 14s - loss: 7.2681e-05 - val_loss: 5.4374e-05 - 14s/epoch - 14ms/step


# Train covariate model

Merge health and mortality data

In [79]:
print(df['age'].unique)

<bound method Series.unique of 0        18-24
1        18-24
2        18-24
3        18-24
4        18-24
         ...  
38849      80+
38850      80+
38851      80+
38852      80+
38853      80+
Name: age, Length: 37570, dtype: category
Categories (13, object): ['18-24', '25-29', '30-34', '35-39', ..., '65-69', '70-74', '75-79', '80+']>


In [97]:
combined_data = []
for i, row in enumerate(data):
    state = row[0]
    gender = row[1]
    year = row[2]
    age = row[3]
    
    #print(f"row {i}")
    #print(f"row printed: {row}")

    matching = np.logical_and(health_data[:, 0] == state, health_data[:, 2] == gender)
    matching = np.logical_and(matching, health_data[:, 3] == year)
    if 18 <= age <= 24:
        matching = np.logical_and(matching, health_data[:, 1] == '18-24')
    if 25 <= age <= 29:
        matching = np.logical_and(matching, health_data[:, 1] == '25-29')
    if 30 <= age <= 34:
        matching = np.logical_and(matching, health_data[:, 1] == '30-34')
    if 35 <= age <= 39:
        matching = np.logical_and(matching, health_data[:, 1] == '35-39')
    if 40 <= age <= 44:
        matching = np.logical_and(matching, health_data[:, 1] == '40-44')
    if 45 <= age <= 49:
        matching = np.logical_and(matching, health_data[:, 1] == '45-49')
    if 50 <= age <= 54:
        matching = np.logical_and(matching, health_data[:, 1] == '50-54')
    if 55 <= age <= 59:
        matching = np.logical_and(matching, health_data[:, 1] == '55-59')
    if 60 <= age <= 64:
        matching = np.logical_and(matching, health_data[:, 1] == '60-64')
    if 65 <= age <= 69:
        matching = np.logical_and(matching, health_data[:, 1] == '65-69')
    if 70 <= age <= 74:
        matching = np.logical_and(matching, health_data[:, 1] == '70-74')
    if 75 <= age <= 79:
        matching = np.logical_and(matching, health_data[:, 1] == '75-79')
    if age >= 80:
        matching = np.logical_and(matching, health_data[:, 1] == '80+')
    # print(health_data[matching])
    try:
        bmi = health_data[matching][:,4][0]
        combined_row = [state, gender, year, age, bmi, row[4]]
        combined_data.append(combined_row)
    except IndexError:
        print(f"Failed for row {i}: {row}")
        combined_row = [state, gender, year, age, np.nan, row[4]]  # Using np.nan as a placeholder for missing BMI
        combined_data.append(combined_row)
        continue 


combined_data = np.array(combined_data)
print(combined_data)

Failed for row 18368: [8.000e+00 0.000e+00 1.993e+03 1.800e+01 4.800e-04]
Failed for row 18369: [8.000e+00 0.000e+00 1.993e+03 1.900e+01 6.000e-04]
Failed for row 18370: [8.000e+00 0.000e+00 1.993e+03 2.000e+01 4.600e-04]
Failed for row 18371: [8.000e+00 0.000e+00 1.993e+03 2.100e+01 5.400e-04]
Failed for row 18372: [8.000e+00 0.000e+00 1.993e+03 2.200e+01 6.100e-04]
Failed for row 18373: [8.000e+00 0.000e+00 1.993e+03 2.300e+01 7.200e-04]
Failed for row 18374: [8.000e+00 0.000e+00 1.993e+03 2.400e+01 8.400e-04]
Failed for row 18375: [8.000e+00 0.000e+00 1.993e+03 2.500e+01 5.900e-04]
Failed for row 18376: [8.000e+00 0.000e+00 1.993e+03 2.600e+01 8.300e-04]
Failed for row 18377: [8.000e+00 0.000e+00 1.993e+03 2.700e+01 8.700e-04]
Failed for row 18378: [8.000e+00 0.000e+00 1.993e+03 2.800e+01 9.300e-04]
Failed for row 18379: [8.000e+00 0.000e+00 1.993e+03 2.900e+01 8.500e-04]
Failed for row 18380: [8.000e+00 0.000e+00 1.993e+03 3.000e+01 8.000e-04]
Failed for row 18381: [8.000e+00 0.000

KeyboardInterrupt: 